# 3-qubit teleportation with gate + T1/Tphi noise

We extend the feed-forward teleportation example with depolarizing gate errors, measurement flips, and explicit amplitude-damping (T1) plus pure dephasing (Tphi) channels. Tweak the parameters below to mirror a target device.


In [1]:
import math

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity
from qiskit_aer import AerSimulator
from qiskit_aer.noise import (
    NoiseModel,
    ReadoutError,
    amplitude_damping_error,
    depolarizing_error,
    phase_damping_error,
)


## Noise building blocks
The helpers below assemble amplitude/phase damping channels and combine them
with depolarizing plus readout errors into a reusable `NoiseModel`.


In [2]:
def combined_damping_error(t1=None, tphi=None, gate_time=2e-7):
    """Return a single-qubit error channel for amplitude damping (T1) and pure dephasing (Tphi)."""
    errors = []
    if t1:
        p_amp = 1 - math.exp(-gate_time / t1)
        errors.append(amplitude_damping_error(p_amp))
    if tphi:
        p_phase = 1 - math.exp(-gate_time / tphi)
        errors.append(phase_damping_error(p_phase))

    if not errors:
        return None

    combined = errors[0]
    for err in errors[1:]:
        combined = combined.compose(err)
    return combined


In [3]:
def build_gate_noise_model(
    p_single=1e-3,
    p_two=5e-3,
    p_readout=1.5e-2,
    t1=150e-6,
    tphi=200e-6,
    gate_time=2e-7,
):
    noise_model = NoiseModel()

    single_depolar = depolarizing_error(p_single, 1)
    two_depolar = depolarizing_error(p_two, 2)
    readout_error = ReadoutError([[1 - p_readout, p_readout], [p_readout, 1 - p_readout]])

    damping = combined_damping_error(t1=t1, tphi=tphi, gate_time=gate_time)
    if damping is not None:
        single_quantum_error = single_depolar.compose(damping)
        two_quantum_error = two_depolar.compose(damping.tensor(damping))
    else:
        single_quantum_error = single_depolar
        two_quantum_error = two_depolar

    single_gate_set = ["id", "x", "sx", "rz", "h"]
    two_gate_set = ["cx", "cz"]

    noise_model.add_all_qubit_quantum_error(single_quantum_error, single_gate_set)
    noise_model.add_all_qubit_quantum_error(two_quantum_error, two_gate_set)
    noise_model.add_all_qubit_readout_error(readout_error)
    return noise_model


In [4]:
def original_pair_circuit():
    circ = QuantumCircuit(3)
    circ.h(0)
    circ.cx(0, 1)
    circ.save_density_matrix(label="rho_orig")
    return circ


def teleportation_circuit():
    tele_circ = QuantumCircuit(3, 1)
    tele_circ.h(0)
    tele_circ.cx(0, 1)
    tele_circ.h(2)
    tele_circ.cz(1, 2)
    tele_circ.h(1)
    tele_circ.measure(1, 0)
    # Apply the feed-forward X correction on q2 only if the X-basis outcome is s = 1
    with tele_circ.if_test((tele_circ.cregs[0], 1)):
        tele_circ.x(2)
    tele_circ.h(2)
    tele_circ.save_density_matrix(label="rho_final")
    return tele_circ


def simulate_density_matrix(circuit, simulator, label):
    compiled = transpile(circuit, simulator)
    result = simulator.run(compiled).result()
    return DensityMatrix(result.data(0)[label])


## Baseline noisy teleportation
Instantiate the ideal and noisy simulators once, compute the reduced density
matrices for `(q0,q1)` and `(q0,q2)`, and print their fidelities for quick
reference.


In [5]:
# Representative coherence and gate error values (adjust to match hardware)
noise_params = {
    "p_single": 2e-3,
    "p_two": 1e-2,
    "p_readout": 1.5e-2,
    "t1": 120e-6,      # seconds
    "tphi": 180e-6,    # seconds
    "gate_time": 250e-9,
}

ideal_sim = AerSimulator(method="density_matrix")
noise_model = build_gate_noise_model(**noise_params)
noisy_sim = AerSimulator(method="density_matrix", noise_model=noise_model)

rho_full_orig = simulate_density_matrix(original_pair_circuit(), ideal_sim, "rho_orig")
rho_01 = partial_trace(rho_full_orig, [2])

rho_full_tele_ideal = simulate_density_matrix(teleportation_circuit(), ideal_sim, "rho_final")
rho_02_ideal = partial_trace(rho_full_tele_ideal, [1])

rho_full_tele_noisy = simulate_density_matrix(teleportation_circuit(), noisy_sim, "rho_final")
rho_02_noisy = partial_trace(rho_full_tele_noisy, [1])

print("Noise parameters:")
for k, v in noise_params.items():
    print(f"  {k}: {v}")
print("\nFidelity (ideal teleportation vs original pair):", state_fidelity(rho_01, rho_02_ideal))
print("Fidelity (noisy teleportation vs original pair):", state_fidelity(rho_01, rho_02_noisy))


Noise parameters:
  p_single: 0.002
  p_two: 0.01
  p_readout: 0.015
  t1: 0.00012
  tphi: 0.00018
  gate_time: 2.5e-07

Fidelity (ideal teleportation vs original pair): 1.0000000000000004
Fidelity (noisy teleportation vs original pair): 0.9507431790312757


## Coherence sweep
Vary `T1`/`Tphi` to see how reduced coherence times degrade the teleported pair
while leaving the depolarizing and readout parameters fixed.


In [6]:
print("\nT1 / dephasing sweep (shorter coherence -> lower fidelity):")
for t1_us in [80, 120, 200]:
    params = dict(noise_params)
    # Convert the user-friendly microsecond inputs into seconds for the model
    params["t1"] = t1_us * 1e-6
    params["tphi"] = (t1_us + 40) * 1e-6
    nm = build_gate_noise_model(**params)
    sim = AerSimulator(method="density_matrix", noise_model=nm)
    rho_tele = partial_trace(simulate_density_matrix(teleportation_circuit(), sim, "rho_final"), [1])
    fidelity = state_fidelity(rho_01, rho_tele)
    print(f"  T1={t1_us:>3} us, Tphi={t1_us + 40:>3} us -> fidelity={fidelity:.6f}")



T1 / dephasing sweep (shorter coherence -> lower fidelity):
  T1= 80 us, Tphi=120 us -> fidelity=0.957318
  T1=120 us, Tphi=160 us -> fidelity=0.956971
  T1=200 us, Tphi=240 us -> fidelity=0.954116


## Takeaways
The fidelity varies only slightly across the sampled coherence windows, showing
that depolarizing and measurement errors dominate once T1/Tphi exceed ~100 μs
for this short-depth circuit.
